<center>


# Curso *Ingeniería de Características*
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>

## Descargando datos en XML

Daniel Eduardo Vázquez Espinoza


</center>

### **Objetivos:**
  

*   Descargar un documento XML de wikipedia y analizar el arbol que se genera
*   Definir un data grame en funcion del analisis que pueda ser util para algun proyecto se ciencia de datos.
*  Desarrolla un script que genere el DataFrame definido en el punto anterior
*   Guarda el DataFrame generado como un archivo de tipo parquet




Comenzaremos importando las librerias necesarias:

In [5]:
import os #libreria para manipular directorios
import urllib.request  #Descarga de datos
import xml.etree.ElementTree as et
import datetime
import pandas as pd
import numpy as np
import zipfile
import re #Para utilizar expresiones regulares

Descargamos el archivo desde su enlace y guardamos en el path

In [6]:
enlace = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"
nombre = "poetas.xml"
subdir = "./data/"

if not os.path.exists(subdir + nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(enlace, subdir + nombre)

Al desplegar su estrutura se puede apreciar que se solo cambia el nombre del atributo al final : 

"{ http://www.mediawiki.org/xml/export-0.10/ }Atributo"

In [7]:
poetas=et.parse(subdir+ nombre)
root = poetas.getroot()

In [8]:
[elem.tag for elem in root.iter()][50:60]

['{http://www.mediawiki.org/xml/export-0.10/}id',
 '{http://www.mediawiki.org/xml/export-0.10/}comment',
 '{http://www.mediawiki.org/xml/export-0.10/}model',
 '{http://www.mediawiki.org/xml/export-0.10/}format',
 '{http://www.mediawiki.org/xml/export-0.10/}text',
 '{http://www.mediawiki.org/xml/export-0.10/}sha1',
 '{http://www.mediawiki.org/xml/export-0.10/}page',
 '{http://www.mediawiki.org/xml/export-0.10/}title',
 '{http://www.mediawiki.org/xml/export-0.10/}ns',
 '{http://www.mediawiki.org/xml/export-0.10/}id']

Accedemos al atributo autor, su biografía, fechas de modificación del artículo de wikipedia, el nombre del usuario que editó el artículo.

Ponemos esta información en una tabla de pandas.


In [9]:
autores = []
for iautor in root.iter('{http://www.mediawiki.org/xml/export-0.10/}title'):
    autores.append(iautor.text)

fechas_modificacion = []
for imodi in root.iter('{http://www.mediawiki.org/xml/export-0.10/}timestamp'):
    fechas_modificacion.append(imodi.text)

biography = []
for ibio in root.iter('{http://www.mediawiki.org/xml/export-0.10/}text'):
    biography.append(ibio.text)
username=[]
for iuser in root.iter('{http://www.mediawiki.org/xml/export-0.10/}username'):
    username.append(iuser.text)
    

In [10]:
data = pd.DataFrame(columns = ['Autor','Biografia','Fecha_Modificacion'])
for i in range(len(autores)):
  data = pd.concat([data,
                  pd.DataFrame.from_records([{'Autor': autores[i], 
                                              'Biografia': biography[i], 
                                              "Fecha_Modificacion": fechas_modificacion[i]}]
                                            )], ignore_index=True)

In [11]:
year_birth=[]
day_month_birth=[]
for i in range(len(data)):
  
  try:
    year_birth.append(re.search(r"\[\[([1-3][0-9]{3})\]\]", data["Biografia"].iloc[i]).group(1))
  except:
    year_birth.append(np.nan)
  
  try:
    day_month_birth.append(re.search(r"\[\[([1-3][1-9]\s.*?)\]\]", data["Biografia"].iloc[i]).group(1))
  except:
    day_month_birth.append(np.nan)

In [12]:
data["Año_Nacimiento"]=year_birth
data["Dia_Mes_Nacimiento"]=day_month_birth
data

,Autor,Biografia,Fecha_Modificacion,Año_Nacimiento,Dia_Mes_Nacimiento
0,Julia Morilla de Campbell,'''Julia Morilla de Campbell''' ([[Rosario (Ar...,2017-10-30T07:46:11Z,1922,18 de julio
1,Luis Negreti,{{Ficha de escritor\n|Imagen = NE...,2016-09-17T17:35:26Z,1890,16 de enero
2,Poldy Bird,{{Ficha de persona\n| padres = Enrique Bird Mo...,2018-06-11T04:34:41Z,1941,16 de diciembre
3,Ana María Shua,{{Ficha de persona\n|imagen=\n|nombre de nacim...,2018-06-13T18:50:17Z,1951,22 de abril
4,León Benarós,{{Ficha de persona\n|nombre = León B...,2018-04-29T21:49:56Z,1915,25 de agosto
...,...,...,...,...,...
634,Humberto Tejera,{{Ficha de persona\n| nombre = Humberto Tejera...,2018-07-06T17:07:38Z,NaN,NaN
635,Mario Molina Cruz,{{Ficha de escritor\n|nombre= Mario Molina Cru...,2018-07-23T14:27:14Z,1955,19 de enero
636,Luis Ignacio Helguera,{{Ficha de persona\n| nombre = Luis Ignacio He...,2018-05-11T14:19:35Z,1962,11 de mayo
637,Daniel Olivares Viniegra,{{Promocional|8|julio}}\n\n'''Daniel Olivares ...,2018-07-20T21:56:20Z,NaN,NaN


Con la informacion de esta tabla se podria realizar un analisis del año en que hubo mas nacimientos de autores, analizar si los sucesos o eventos que sucedieron en las fechas con mas concentracion de nacimientos son reelevantes.

In [13]:
data.to_parquet('poetas.parquet.gzip',compression='gzip')